## P1

In [8]:
sum = 0

for x in range(1,1000):
    if x % 3 == 0 or x % 5 ==0:
        print(x)
        sum+=x

3
5
6
9
10
12
15
18
20
21
24
25
27
30
33
35
36
39
40
42
45
48
50
51
54
55
57
60
63
65
66
69
70
72
75
78
80
81
84
85
87
90
93
95
96
99
100
102
105
108
110
111
114
115
117
120
123
125
126
129
130
132
135
138
140
141
144
145
147
150
153
155
156
159
160
162
165
168
170
171
174
175
177
180
183
185
186
189
190
192
195
198
200
201
204
205
207
210
213
215
216
219
220
222
225
228
230
231
234
235
237
240
243
245
246
249
250
252
255
258
260
261
264
265
267
270
273
275
276
279
280
282
285
288
290
291
294
295
297
300
303
305
306
309
310
312
315
318
320
321
324
325
327
330
333
335
336
339
340
342
345
348
350
351
354
355
357
360
363
365
366
369
370
372
375
378
380
381
384
385
387
390
393
395
396
399
400
402
405
408
410
411
414
415
417
420
423
425
426
429
430
432
435
438
440
441
444
445
447
450
453
455
456
459
460
462
465
468
470
471
474
475
477
480
483
485
486
489
490
492
495
498
500
501
504
505
507
510
513
515
516
519
520
522
525
528
530
531
534
535
537
540
543
545
546
549
550
552
555
558
560
561
56

233168

## P2

In [32]:
import numpy as np
def gen_Fibonacci(upper_limit):
    nums = []
    nums.append(1)
    nums.append(2)
    while nums[-2]+nums[-1] <= upper_limit:
        nums.append(nums[-2]+nums[-1])
    return nums

nums = gen_Fibonacci(4e6)

sum = 0
for x in nums:
    if x % 2 == 0:
        print(x)
        sum += x
sum

2
8
34
144
610
2584
10946
46368
196418
832040
3524578


4613732

## P3

In [137]:
def ck_prime(num):
    x = 2
    while num % x != 0:
        x+=1
#     is_prime = any((num % np.array(range(2,num)))==0)==False ## Vectorized version (memory issue)
    return(x==num)

def find_max_prime(num):
    for x in range(2,600851475143+1):
#         if x % 10000 == 0:
#             print(x)
        if num % x == 0:
            if ck_prime(num / x):
                return(int(num / x))
        
find_max_prime(600851475143)

6857

## P64

In [45]:
import numpy as np
def next_set(N,b,c):
    """
    Helper function to get next set based on N, bn and cn
    For the special case of initialization, set b0 = np.floor(np.sqrt(N)) and c0 = 1
    """
    rt = np.floor(np.sqrt(N))
    c2 = (N - b**2)/c
    a2 = np.floor((b+rt)/c2)
    b2 = a2*c2 -b
    assert(c2.is_integer())
    return(int(a2),int(b2),int(c2))

def get_period(N):
    if np.sqrt(N).is_integer():
        return(0)
    else:
        a1,b1,c1 = next_set(N,np.floor(np.sqrt(N)),1)
        i=1
        an,bn,cn = next_set(N,b1,c1)
        while (an,bn,cn) != (a1,b1,c1):
            an,bn,cn = next_set(N,bn,cn)
            i+=1
        return(i)
    
store = []
for N in range(1,10001):
    if get_period(N) % 2 == 1:
        store.append(N)
print(len(store))

## P252

In [5]:
import numpy as np
def rand_gen(N):
    """Generate randome numbers"""
    T = []
    Sn = 290797
    for n in range(N):
        Sn = Sn**2 % 50515093
        T.append((Sn % 2000) - 1000)
    return T
randoms = np.array(rand_gen(1000))
dots = randoms.reshape([-1,2])
print(dots[:2])

[[ 527  144]
 [-488  732]]


In [49]:
def get_vector(dot1,dot2):
    x = dot2[0]-dot1[0]
    y = dot2[1]-dot1[1]
    return([x,y])
    
def get_angle_area(v1,v2):
    v1_mag = np.linalg.norm(v1)
    v2_mag = np.linalg.norm(v2)
    
    cos_theta = np.dot(v1,v2)/v1_mag/v2_mag
    sin_theta = np.cross(v1,v2)/v1_mag/v2_mag
    if sin_theta>0:
        angle = np.rad2deg(np.arccos(cos_theta))
    else:
        angle = 360 - np.rad2deg(np.arccos(cos_theta))
    area = np.cross(v1,v2).tolist()/2
    return(angle,area)

global maxArea
maxArea = 0


In [50]:
v1 = get_vector(dots[0],dots[1])
v2 = get_vector(dots[1],dots[2])
print(v1,v2)
print(np.dot(v1,v2))
print(np.cross(v1,v2))

[-1015, 588] [34, -1679]
-1021762
1684193


In [51]:
get_angle_area([1,0],[1,1])

(45.00000000000001, 0.5)

1684193

In [18]:

def get_curve(dot1,dot2):
    x1,y1 = dot1[0],dot1[1]
    x2,y2 = dot2[0],dot2[1]
    k = (y1-y2)/(x1-x2)
    b = y1 - k*x1
    return(k,b)
line_const(dots[0],dots[1])

def is_valid(dots_included,dot_new,dots_else):
    

(-0.5793103448275863, 449.29655172413794)

In [10]:
def mul(N):
    if N==1:
        return(1)
    else:
        return(int(N*mul(N-1)))

In [12]:
mul(500)

1220136825991110068701238785423046926253574342803192842192413588385845373153881997605496447502203281863013616477148203584163378722078177200480785205159329285477907571939330603772960859086270429174547882424912726344305670173270769461062802310452644218878789465754777149863494367781037644274033827365397471386477878495438489595537537990423241061271326984327745715546309977202781014561081188373709531016356324432987029563896628911658974769572087926928871281780070265174507768410719624390394322536422605234945850129918571501248706961568141625359056693423813008856249246891564126775654481886506593847951775360894005745238940335798476363944905313062323749066445048824665075946735862074637925184200459369692981022263971952597190945217823331756934581508552332820762820023402626907898342451712006207714640979456116127629145951237229913340169552363850942885592018727433795173014586357570828355780158735432768888680120399882384702151467605445407663535984174430480128938313896881639487469658817504506926365338175